In [1]:
import warnings
warnings.filterwarnings('ignore')

import pandas as pd
import numpy as np

from sklearn.pipeline import Pipeline,FeatureUnion
from sklearn.base import BaseEstimator,TransformerMixin

from mypipes import *

In [ ]:
train_file=r'C:\Users\chirag\Desktop\ML IITK\ML Intro\DATA\loan_data_train.csv'
test_file=r'C:\Users\chirag\Desktop\ML IITK\ML Intro\DATA\loan_data_test.csv'

ld_train=pd.read_csv(train_file)
ld_test=pd.read_csv(test_file)

In [3]:
ld_train.head()

,ID,Amount.Requested,Amount.Funded.By.Investors,Interest.Rate,Loan.Length,Loan.Purpose,Debt.To.Income.Ratio,State,Home.Ownership,Monthly.Income,FICO.Range,Open.CREDIT.Lines,Revolving.CREDIT.Balance,Inquiries.in.the.Last.6.Months,Employment.Length
0,79542.0,25000,25000,18.49%,60 months,debt_consolidation,27.56%,VA,MORTGAGE,8606.56,720-724,11,15210,3.0,5 years
1,75473.0,19750,19750,17.27%,60 months,debt_consolidation,13.39%,NY,MORTGAGE,6737.50,710-714,14,19070,3.0,4 years
2,67265.0,2100,2100,14.33%,36 months,major_purchase,3.50%,LA,OWN,1000.00,690-694,13,893,1.0,< 1 year
3,80167.0,28000,28000,16.29%,36 months,credit_card,19.62%,NV,MORTGAGE,7083.33,710-714,12,38194,1.0,10+ years
4,17240.0,24250,17431.82,12.23%,60 months,credit_card,23.79%,OH,MORTGAGE,5833.33,730-734,6,31061,2.0,10+ years


In [4]:
ld_test.head()

,ID,Amount.Requested,Amount.Funded.By.Investors,Loan.Length,Loan.Purpose,Debt.To.Income.Ratio,State,Home.Ownership,Monthly.Income,FICO.Range,Open.CREDIT.Lines,Revolving.CREDIT.Balance,Inquiries.in.the.Last.6.Months,Employment.Length
0,20093,5000,5000,60 months,moving,12.59%,NY,RENT,4416.67,690-694,13,7686,0,< 1 year
1,62445,18000,18000,60 months,debt_consolidation,4.93%,CA,RENT,5258.50,710-714,6,11596,0,10+ years
2,65248,7200,7200,60 months,debt_consolidation,25.16%,LA,MORTGAGE,3750.00,750-754,13,7283,0,6 years
3,81822,7200,7200,36 months,debt_consolidation,17.27%,NY,MORTGAGE,3416.67,790-794,14,4838,0,10+ years
4,57923,22000,22000,60 months,debt_consolidation,18.28%,MI,MORTGAGE,6083.33,720-724,9,20181,0,8 years


In [5]:
ld_train.sample(10)

,ID,Amount.Requested,Amount.Funded.By.Investors,Interest.Rate,Loan.Length,Loan.Purpose,Debt.To.Income.Ratio,State,Home.Ownership,Monthly.Income,FICO.Range,Open.CREDIT.Lines,Revolving.CREDIT.Balance,Inquiries.in.the.Last.6.Months,Employment.Length
532,103512.0,8000,7725,10.99%,36 months,credit_card,13.40%,FL,MORTGAGE,7000.00,735-739,3,175087,1.0,10+ years
284,77887.0,7000,7000,8.90%,36 months,home_improvement,14.69%,KY,MORTGAGE,5416.67,720-724,14,15272,1.0,3 years
1379,82729.0,12000,12000,10.16%,36 months,debt_consolidation,11.93%,CA,RENT,8054.50,710-714,13,14986,0.0,10+ years
1235,28305.0,5000,5000,17.49%,60 months,car,16.71%,PA,RENT,2400.00,670-674,6,2023,1.0,3 years
1761,49956.0,32000,32000,16.29%,60 months,debt_consolidation,15.14%,AZ,RENT,7055.00,725-729,8,16174,1.0,6 years
190,91334.0,35000,35000,15.31%,36 months,debt_consolidation,19.59%,PA,MORTGAGE,7500.00,695-699,11,29953,2.0,1 year
816,48356.0,9000,9000,6.03%,36 months,major_purchase,9.29%,NY,MORTGAGE,14166.67,770-774,13,40909,0.0,4 years
1442,60832.0,7200,7200,7.90%,36 months,debt_consolidation,21.74%,TX,RENT,4300.00,710-714,7,33189,0.0,10+ years
1676,32181.0,NaN,10000,11.71%,36 months,moving,2.18%,CA,RENT,5500.00,720-724,3,5623,0.0,4 years
536,45225.0,19200,19200,13.11%,60 months,other,3.45%,IL,MORTGAGE,8333.33,745-749,14,50625,3.0,2 years


In [6]:
# don't need : Id,Amount.Funded.By.Investors,Interest.Rate(target var)
# v1, Amount requested : convert it to numeric, treat missing values impute them with median
# v3, Open.CREDIT.Lines : convert it to numeric, treat missing values impute them with median
# v4, Revolving.CREDIT.Balance : convert it to numeric, treat missing values impute them with median

# v5, Debt.To.Income.Ratio : remove percentage sign, convert to numeric and impute with median

# v6-v10, Loan.Length,Loan.Purpose,State,Home.Ownership,Employment.Length : create dummies for these considering frequency cutoff as 20
# ignoring categories having less frequency in data

# v11-v12, Monthly.Income, Inquiries.in.the.Last.6.Months : impute missing values with median

# v13, FICO.Range : split the column at '-', convert the resulting columns to numeric(say a,b)
# then create a new column (a+b)/2

In [7]:
p1=pdPipeline([
    ('var_select',VarSelector(['Amount.Requested','Open.CREDIT.Lines','Revolving.CREDIT.Balance'])),
    ('convert_to_numeric',convert_to_numeric()),
    ('missing_trt',DataFrameImputer())
])

p2=pdPipeline([
    ('var_select',VarSelector(['Debt.To.Income.Ratio'])),
    ('string_clean',string_clean(replace_it='%',replace_with='')),
    ('convert_to_numeric',convert_to_numeric()),
    ('missing_trt',DataFrameImputer())
])

p3=pdPipeline([
    ('var_select',VarSelector(['Loan.Length','Loan.Purpose','State','Home.Ownership','Employment.Length'])),
    ('missing_trt',DataFrameImputer()),
    ('create_dummies',get_dummies_Pipe(20))
])

p4=pdPipeline([
    ('var_select',VarSelector(['Monthly.Income','Inquiries.in.the.Last.6.Months'])),
    ('missing_trt',DataFrameImputer()),
])

p5=pdPipeline([
    ('var_select',VarSelector(['FICO.Range'])),
    ('custom_fico',custom_fico())
])

data_pipe=FeatureUnion([
    ('obj_to_num',p1),
    ('dtlr',p2),
    ('obj_to_dum',p3),
    ('num',p4),
    ('fico',p5)
])

In [8]:
x_train=pd.DataFrame(data=data_pipe.fit_transform(ld_train),
                    columns=data_pipe.get_feature_names())

In [9]:
x_train.shape

(2200, 60)

In [10]:
x_test=pd.DataFrame(data=data_pipe.transform(ld_test),
                   columns=data_pipe.get_feature_names())

In [11]:
x_test.shape

(300, 60)

In [12]:
y_train=ld_train['Interest.Rate'].str.replace('%','').astype(float)

In [13]:
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import cross_val_score

In [14]:
ln=LinearRegression()

In [15]:
cv_mae=-cross_val_score(ln,
                      x_train,y_train,
                      cv=10,
                      scoring='neg_mean_absolute_error')

In [16]:
cv_mae

array([1.74273801, 1.76502624, 1.77011354, 1.65406589, 1.43720296,
       1.63540955, 1.44255408, 1.58043799, 1.52220714, 1.65823135])

In [17]:
cv_mae.mean()

1.6207986760159065

In [18]:
cv_mae.std()

0.11726675023087343

In [19]:
 ln.fit(x_train,y_train)

LinearRegression()

In [20]:
ln.intercept_

73.16432302812574

In [21]:
ln.coef_

array([ 1.60913878e-04, -3.72164153e-02, -3.05844966e-06,  1.88427404e-04,
        1.30627284e+00,  4.47755390e+00, -7.84079703e-01, -8.91888331e-01,
        3.20936459e-02, -6.95275775e-01, -4.22744797e-01, -2.64224120e-01,
       -5.41274155e-01, -9.69555836e-01, -5.98574359e-01,  7.21470875e-01,
       -1.99766458e-01, -1.88645568e-01, -7.45527597e-02,  4.73923022e-01,
       -4.68212871e-01, -5.71694597e-01, -1.79076702e-01, -3.78950003e-01,
       -1.57191036e-01, -1.08257898e-01, -4.96139885e-01, -1.99177682e-01,
       -7.78964606e-02,  4.10456351e-02, -2.67193758e-01,  4.08575782e-01,
        3.80179542e-01, -9.81649463e-02, -6.21920499e-02, -1.77735228e-01,
       -3.50604056e-01, -2.33745724e-01, -3.92591988e-02,  1.10048502e-02,
        1.87183988e-02,  1.85782841e-01, -2.39881640e+00, -2.17035525e+00,
       -2.06526514e+00,  5.26184321e-01,  3.52218686e-01,  2.87554381e-01,
        1.51007910e-01,  5.83377246e-01,  4.22222352e-01,  2.41210867e-01,
        3.88741640e-01,  

In [22]:
list(zip(x_train.columns,ln.coef_))

[('obj_to_num__Amount.Requested', 0.00016091387809670106),
 ('obj_to_num__Open.CREDIT.Lines', -0.03721641529315641),
 ('obj_to_num__Revolving.CREDIT.Balance', -3.0584496637280838e-06),
 ('dtlr__Debt.To.Income.Ratio', 0.00018842740388802515),
 ('obj_to_dum__Loan.Length_36 months', 1.3062728365133454),
 ('obj_to_dum__Loan.Length_60 months', 4.477553903304561),
 ('obj_to_dum__Loan.Purpose_debt_consolidation', -0.784079702742753),
 ('obj_to_dum__Loan.Purpose_credit_card', -0.8918883314528003),
 ('obj_to_dum__Loan.Purpose_other', 0.03209364593729813),
 ('obj_to_dum__Loan.Purpose_home_improvement', -0.6952757746474028),
 ('obj_to_dum__Loan.Purpose_major_purchase', -0.4227447967589232),
 ('obj_to_dum__Loan.Purpose_small_business', -0.26422412044799154),
 ('obj_to_dum__Loan.Purpose_car', -0.5412741545888299),
 ('obj_to_dum__Loan.Purpose_wedding', -0.9695558362069785),
 ('obj_to_dum__Loan.Purpose_medical', -0.5985743591789384),
 ('obj_to_dum__Loan.Purpose_moving', 0.7214708748508427),
 ('obj_to

In [23]:
test_pred=ln.predict(x_test)

In [24]:
test_pred

array([16.73641244, 15.9822577 , 10.41724451,  3.71534168, 15.21108834,
        7.0351977 , 15.57246453, 10.58352597, 15.94955183, 12.38355726,
        9.9366698 , 15.17209087, 11.53730446, 13.67752736, 13.40134753,
       18.33363766, 10.18969518, 15.79040256, 13.37967692, 14.03563066,
       22.50430604, 17.50946104, 12.12105201, 14.6034318 ,  9.68060525,
       11.39039568, 13.33566534, 19.0210008 , 11.80641539, 16.88384891,
       15.0680717 , 15.29064393, 12.35705772, 15.07255624, 13.78562695,
       14.07528779, 19.52106736, 11.3554855 , 12.00458203, 16.82962478,
       14.15907615, 11.12807774, 14.8967374 , 13.17699098, 15.55418833,
       17.04873439, 14.90876743, 19.10551152, 17.0897673 , 10.33940429,
       13.57931315, 19.75300247, 10.02009378, 19.49728229, 15.57850152,
       15.06801067, 17.10315371, 14.53133523, 10.93874049, 14.48472924,
       13.05230477, 17.09882657,  8.84706185, 14.56313892, 10.50916007,
       11.43247185, 12.79734112, 14.90266567, 12.05952277, 14.46

In [25]:
pd.DataFrame(test_pred).to_csv("mysubmission_linearReg.csv",index=False)

In [26]:
import os
os.getcwd()

'C:\\Users\\chirag\\Desktop\\ML IITK\\ML Intro'


# Ridge Regression

In [27]:
from sklearn.linear_model import Ridge,Lasso
from sklearn.model_selection import GridSearchCV

In [28]:
lambdas=np.linspace(1,100,100)

In [29]:
lambdas

array([  1.,   2.,   3.,   4.,   5.,   6.,   7.,   8.,   9.,  10.,  11.,
        12.,  13.,  14.,  15.,  16.,  17.,  18.,  19.,  20.,  21.,  22.,
        23.,  24.,  25.,  26.,  27.,  28.,  29.,  30.,  31.,  32.,  33.,
        34.,  35.,  36.,  37.,  38.,  39.,  40.,  41.,  42.,  43.,  44.,
        45.,  46.,  47.,  48.,  49.,  50.,  51.,  52.,  53.,  54.,  55.,
        56.,  57.,  58.,  59.,  60.,  61.,  62.,  63.,  64.,  65.,  66.,
        67.,  68.,  69.,  70.,  71.,  72.,  73.,  74.,  75.,  76.,  77.,
        78.,  79.,  80.,  81.,  82.,  83.,  84.,  85.,  86.,  87.,  88.,
        89.,  90.,  91.,  92.,  93.,  94.,  95.,  96.,  97.,  98.,  99.,
       100.])

In [30]:
params={'alpha':lambdas}

In [31]:
params

{'alpha': array([  1.,   2.,   3.,   4.,   5.,   6.,   7.,   8.,   9.,  10.,  11.,
         12.,  13.,  14.,  15.,  16.,  17.,  18.,  19.,  20.,  21.,  22.,
         23.,  24.,  25.,  26.,  27.,  28.,  29.,  30.,  31.,  32.,  33.,
         34.,  35.,  36.,  37.,  38.,  39.,  40.,  41.,  42.,  43.,  44.,
         45.,  46.,  47.,  48.,  49.,  50.,  51.,  52.,  53.,  54.,  55.,
         56.,  57.,  58.,  59.,  60.,  61.,  62.,  63.,  64.,  65.,  66.,
         67.,  68.,  69.,  70.,  71.,  72.,  73.,  74.,  75.,  76.,  77.,
         78.,  79.,  80.,  81.,  82.,  83.,  84.,  85.,  86.,  87.,  88.,
         89.,  90.,  91.,  92.,  93.,  94.,  95.,  96.,  97.,  98.,  99.,
        100.])}

In [32]:
model=Ridge(fit_intercept=True)

In [33]:
grid_search=GridSearchCV(model,
                         param_grid=params,
                         cv=10,
                        scoring='neg_mean_absolute_error',
                        verbose=20,
                        n_jobs=-1)

In [34]:
grid_search.fit(x_train,y_train)

Fitting 10 folds for each of 100 candidates, totalling 1000 fits


GridSearchCV(cv=10, estimator=Ridge(), n_jobs=-1,
             param_grid={'alpha': array([  1.,   2.,   3.,   4.,   5.,   6.,   7.,   8.,   9.,  10.,  11.,
        12.,  13.,  14.,  15.,  16.,  17.,  18.,  19.,  20.,  21.,  22.,
        23.,  24.,  25.,  26.,  27.,  28.,  29.,  30.,  31.,  32.,  33.,
        34.,  35.,  36.,  37.,  38.,  39.,  40.,  41.,  42.,  43.,  44.,
        45.,  46.,  47.,  48.,  49.,  50.,  51.,  52.,  53.,  54.,  55.,
        56.,  57.,  58.,  59.,  60.,  61.,  62.,  63.,  64.,  65.,  66.,
        67.,  68.,  69.,  70.,  71.,  72.,  73.,  74.,  75.,  76.,  77.,
        78.,  79.,  80.,  81.,  82.,  83.,  84.,  85.,  86.,  87.,  88.,
        89.,  90.,  91.,  92.,  93.,  94.,  95.,  96.,  97.,  98.,  99.,
       100.])},
             scoring='neg_mean_absolute_error', verbose=20)

In [35]:
grid_search.best_estimator_

Ridge(alpha=59.0)

In [36]:
grid_search.cv_results_

{'mean_fit_time': array([0.04086809, 0.01936839, 0.01887343, 0.01629038, 0.01497779,
        0.01347091, 0.01660554, 0.01664069, 0.01493711, 0.01352284,
        0.01517444, 0.01513546, 0.01591234, 0.01562688, 0.01527777,
        0.01401622, 0.01306164, 0.01387515, 0.01247728, 0.01388783,
        0.01483209, 0.01466217, 0.01349008, 0.01124794, 0.01503575,
        0.01486397, 0.01542032, 0.01314018, 0.01644392, 0.01271317,
        0.01560929, 0.01543617, 0.0157562 , 0.01439147, 0.01716282,
        0.01604369, 0.01662116, 0.01558187, 0.01885607, 0.01509814,
        0.01676896, 0.01578956, 0.01485858, 0.01501493, 0.0129071 ,
        0.01334889, 0.01491828, 0.01111236, 0.01435809, 0.01474752,
        0.01358964, 0.01131248, 0.00971413, 0.01232839, 0.0115545 ,
        0.01411836, 0.01335719, 0.00962236, 0.01214595, 0.01668491,
        0.01314299, 0.01376679, 0.01572402, 0.01632946, 0.01483583,
        0.01452491, 0.01673269, 0.01538966, 0.01602259, 0.01620123,
        0.01378465, 0.01567199,

In [39]:
def report(results,n_top=3):
    for i in range(1,n_top+1):
        candidates=np.flatnonzero(results['rank_test_score']==i)
        for candidate in candidates:
            print("Model with rank: {0}".format(i))
            print("Mean Validation Score: {0:.6f} (std: {1:.6f})".format(
                    results['mean_test_score'][candidate],
                    results['std_test_score'][candidate]))
            print("Parameters: {0}".format(results['params'][candidate]))
            print("")

In [40]:
report(grid_search.cv_results_,5)

Model with rank: 1
Mean Validation Score: -1.606905 (std: 0.117674)
Parameters: {'alpha': 59.0}

Model with rank: 2
Mean Validation Score: -1.606905 (std: 0.117597)
Parameters: {'alpha': 58.0}

Model with rank: 3
Mean Validation Score: -1.606906 (std: 0.117749)
Parameters: {'alpha': 60.0}

Model with rank: 4
Mean Validation Score: -1.606909 (std: 0.117518)
Parameters: {'alpha': 57.0}

Model with rank: 5
Mean Validation Score: -1.606911 (std: 0.117820)
Parameters: {'alpha': 61.0}



In [41]:
test_pred=grid_search.predict(x_test)

In [42]:
pd.DataFrame(test_pred).to_csv("mySubmission_ridge",index=False)

# For looking at coefficients 

In [43]:
grid_search.best_estimator_

Ridge(alpha=59.0)

In [44]:
ridge_model=grid_search.best_estimator_

In [45]:
ridge_model.fit(x_train,y_train)

Ridge(alpha=59.0)

In [46]:
list(zip(data_pipe.get_feature_names(),ridge_model.coef_))

[('obj_to_num__Amount.Requested', 0.0001659056567176454),
 ('obj_to_num__Open.CREDIT.Lines', -0.037908063378880995),
 ('obj_to_num__Revolving.CREDIT.Balance', -3.1637569333467703e-06),
 ('dtlr__Debt.To.Income.Ratio', -0.0008409375022167036),
 ('obj_to_dum__Loan.Length_36 months', -1.3957035074401458),
 ('obj_to_dum__Loan.Length_60 months', 1.487989718632789),
 ('obj_to_dum__Loan.Purpose_debt_consolidation', -0.32395293779591755),
 ('obj_to_dum__Loan.Purpose_credit_card', -0.3998778113551296),
 ('obj_to_dum__Loan.Purpose_other', 0.3608939940679972),
 ('obj_to_dum__Loan.Purpose_home_improvement', -0.17515395397223982),
 ('obj_to_dum__Loan.Purpose_major_purchase', 0.004410987700190584),
 ('obj_to_dum__Loan.Purpose_small_business', 0.10286757148900036),
 ('obj_to_dum__Loan.Purpose_car', -0.018641873174348147),
 ('obj_to_dum__Loan.Purpose_wedding', -0.21984698028510624),
 ('obj_to_dum__Loan.Purpose_medical', -0.054076034285069456),
 ('obj_to_dum__Loan.Purpose_moving', 0.331535264489298),
 (

In [48]:
ln.coef_/ridge_model.coef_

array([ 9.69911944e-01,  9.81754592e-01,  9.66714488e-01, -2.24068261e-01,
       -9.35924306e-01,  3.00912960e+00,  2.42035065e+00,  2.23040215e+00,
        8.89281796e-02,  3.96951230e+00, -9.58390332e+01, -2.56858519e+00,
        2.90353952e+01,  4.41013943e+00,  1.10691246e+01,  2.17615123e+00,
        2.12437158e+00,  6.18614703e+00, -4.73160592e+00,  1.22902469e+00,
        2.06149061e+00,  2.01208718e+00,  3.26002016e+00,  2.25164303e+00,
        1.72056498e+01,  5.82457716e+00,  2.56508150e+00,  2.48868477e+00,
       -3.13174013e+00,  5.59185867e-01,  4.02365760e+00,  2.11403463e+00,
        1.89664509e+00,  1.90090348e+01, -3.24127331e+00,  6.13615270e+00,
        4.07951368e+00,  3.25241033e+01, -6.76125999e+00,  2.89372796e-01,
        6.07010219e-01,  2.26635930e+00,  1.00887444e+01,  7.63428021e+01,
       -3.71155295e+01,  3.79515587e+00, -1.72319034e+02, -1.16329695e+01,
       -1.06550764e+00,  3.27926770e+00,  7.38888277e+00, -2.16931393e+00,
        1.33033061e+01, -

# Lasso Regression

In [49]:
lambdas=np.linspace(1,10,100)

model=Lasso(fit_intercept=True)

params={'alpha':lambdas}

In [50]:
grid_search=GridSearchCV(model,
                        param_grid=params,
                        cv=10,
                        scoring='neg_mean_absolute_error',
                        verbose=20,n_jobs=-1)

In [51]:
grid_search.fit(x_train,y_train)

Fitting 10 folds for each of 100 candidates, totalling 1000 fits


GridSearchCV(cv=10, estimator=Lasso(), n_jobs=-1,
             param_grid={'alpha': array([ 1.        ,  1.09090909,  1.18181818,  1.27272727,  1.36363636,
        1.45454545,  1.54545455,  1.63636364,  1.72727273,  1.81818182,
        1.90909091,  2.        ,  2.09090909,  2.18181818,  2.27272727,
        2.36363636,  2.45454545,  2.54545455,  2.63636364,  2.72727273,
        2.81818182,  2.90909091,  3.        ,  3.09090909,  3.18181818,
        3.27272727,  3.36363636,  3.4...
        7.36363636,  7.45454545,  7.54545455,  7.63636364,  7.72727273,
        7.81818182,  7.90909091,  8.        ,  8.09090909,  8.18181818,
        8.27272727,  8.36363636,  8.45454545,  8.54545455,  8.63636364,
        8.72727273,  8.81818182,  8.90909091,  9.        ,  9.09090909,
        9.18181818,  9.27272727,  9.36363636,  9.45454545,  9.54545455,
        9.63636364,  9.72727273,  9.81818182,  9.90909091, 10.        ])},
             scoring='neg_mean_absolute_error', verbose=20)

In [52]:
grid_search.best_estimator_

Lasso()

In [53]:
lambdas=np.linspace(0.001,2,100)

params={'alpha':lambdas}

In [54]:
grid_search=GridSearchCV(model,
                        param_grid=params,
                        cv=10,
                        scoring='neg_mean_absolute_error')

In [55]:
grid_search.fit(x_train,y_train)

GridSearchCV(cv=10, estimator=Lasso(),
             param_grid={'alpha': array([1.00000000e-03, 2.11919192e-02, 4.13838384e-02, 6.15757576e-02,
       8.17676768e-02, 1.01959596e-01, 1.22151515e-01, 1.42343434e-01,
       1.62535354e-01, 1.82727273e-01, 2.02919192e-01, 2.23111111e-01,
       2.43303030e-01, 2.63494949e-01, 2.83686869e-01, 3.03878788e-01,
       3.24070707e-01, 3.44262626e-01, 3.64454545e-01, 3...
       1.53558586e+00, 1.55577778e+00, 1.57596970e+00, 1.59616162e+00,
       1.61635354e+00, 1.63654545e+00, 1.65673737e+00, 1.67692929e+00,
       1.69712121e+00, 1.71731313e+00, 1.73750505e+00, 1.75769697e+00,
       1.77788889e+00, 1.79808081e+00, 1.81827273e+00, 1.83846465e+00,
       1.85865657e+00, 1.87884848e+00, 1.89904040e+00, 1.91923232e+00,
       1.93942424e+00, 1.95961616e+00, 1.97980808e+00, 2.00000000e+00])},
             scoring='neg_mean_absolute_error')

In [56]:
grid_search.best_estimator_

Lasso(alpha=0.021191919191919192)

In [57]:
report(grid_search.cv_results_,5)

Model with rank: 1
Mean Validation Score: -1.600706 (std: 0.123574)
Parameters: {'alpha': 0.021191919191919192}

Model with rank: 2
Mean Validation Score: -1.608777 (std: 0.129103)
Parameters: {'alpha': 0.041383838383838384}

Model with rank: 3
Mean Validation Score: -1.615683 (std: 0.129650)
Parameters: {'alpha': 0.061575757575757575}

Model with rank: 4
Mean Validation Score: -1.616717 (std: 0.115096)
Parameters: {'alpha': 0.001}

Model with rank: 5
Mean Validation Score: -1.620303 (std: 0.130280)
Parameters: {'alpha': 0.08176767676767677}



In [58]:
lasso_model=grid_search.best_estimator_

In [59]:
lasso_model.fit(x_train,y_train)

Lasso(alpha=0.021191919191919192)

In [60]:
lasso_model.intercept_

72.05485736242387

In [62]:
list(zip(data_pipe.get_feature_names(),lasso_model.coef_))

[('obj_to_num__Amount.Requested', 0.00016002823754526234),
 ('obj_to_num__Open.CREDIT.Lines', -0.0380807516599849),
 ('obj_to_num__Revolving.CREDIT.Balance', -3.190939063194295e-06),
 ('dtlr__Debt.To.Income.Ratio', -0.00035647063732539624),
 ('obj_to_dum__Loan.Length_36 months', -0.0),
 ('obj_to_dum__Loan.Length_60 months', 3.03420127536005),
 ('obj_to_dum__Loan.Purpose_debt_consolidation', -0.17001965888491824),
 ('obj_to_dum__Loan.Purpose_credit_card', -0.2136884995089826),
 ('obj_to_dum__Loan.Purpose_other', 0.31957133590263687),
 ('obj_to_dum__Loan.Purpose_home_improvement', -0.0),
 ('obj_to_dum__Loan.Purpose_major_purchase', 0.0),
 ('obj_to_dum__Loan.Purpose_small_business', 0.0),
 ('obj_to_dum__Loan.Purpose_car', 0.0),
 ('obj_to_dum__Loan.Purpose_wedding', -0.0),
 ('obj_to_dum__Loan.Purpose_medical', -0.0),
 ('obj_to_dum__Loan.Purpose_moving', 0.0),
 ('obj_to_dum__State_CA', -0.0),
 ('obj_to_dum__State_NY', 0.0),
 ('obj_to_dum__State_FL', 0.0),
 ('obj_to_dum__State_TX', 0.2483648

In [63]:
(lasso_model.coef_==0).sum()

46